<h2 align='center'>ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import mlflow.sklearn
import mlflow.xgboost
from mlflow import MlflowClient
from mlflow import MlflowException
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [100]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

2025/01/21 13:52:16 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: RESOURCE_DOES_NOT_EXIST: Could not find experiment with ID 737723914328149319


              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [101]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

2025/01/21 13:52:21 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: RESOURCE_DOES_NOT_EXIST: Could not find experiment with ID 737723914328149319


              precision    recall  f1-score   support

           0       0.96      0.99      0.98       270
           1       0.91      0.67      0.77        30

    accuracy                           0.96       300
   macro avg       0.94      0.83      0.87       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [102]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

2025/01/21 13:52:24 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: RESOURCE_DOES_NOT_EXIST: Could not find experiment with ID 737723914328149319


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [104]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

2025/01/21 13:52:31 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: RESOURCE_DOES_NOT_EXIST: Could not find experiment with ID 737723914328149319


              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [5]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [8]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [9]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9640287769784173,
   'recall': 0.9925925925925926,
   'f1-score': 0.9781021897810219,
   'support': 270.0},
  '1': {'precision': 0.9090909090909091,
   'recall': 0.6666666666666666,
   'f1-score': 0.7692307692307693,
   'support': 30.0},
  'accuracy': 0.96,
  'macro avg': {'precision': 0.9365598430346631,
   'recall': 0.8296296296296296,
   'f1-score': 0.8736664795058956,
   'support': 300.0},
  'weighted avg': {'precision': 0.95

In [10]:
client = MlflowClient(tracking_uri="http://localhost:5001/")

In [11]:
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/627497453440672367', creation_time=1737464077121, experiment_id='627497453440672367', last_update_time=1737464077121, lifecycle_stage='active', name='Anomaly Detection', tags={'mlflow.note.content': 'Datacraft Demo Test of feeding data to MLflow'}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1737463898269, experiment_id='0', last_update_time=1737463898269, lifecycle_stage='active', name='Default', tags={}>]


In [12]:
# Experiment description
experiment_description = (
    "Datacraft Demo "
    "Test of feeding data to MLflow"
)
experiment_tags = {
    "mlflow.note.content": experiment_description,
}

# Create the Experiment
try:
    scoring_experiment = client.create_experiment(
        name="Anomaly Detection",
        tags=experiment_tags,
        #artifact_location="file:./artifacts"  
    )
except MlflowException as e:
    print(e)

RESOURCE_ALREADY_EXISTS: Experiment 'Anomaly Detection' already exists.


In [14]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5001")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/01/21 13:58:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/01/21 13:58:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:5001/#/experiments/627497453440672367/runs/d1e381e14b804d4faf0ad4d2243d102f.
2025/01/21 13:58:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/627497453440672367.
2025/01/21 13:58:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/01/21 13:58:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:5001/#/experiments/627497453440672367/runs/44fdadb916aa45cfa7ce74721dd1348b.
2025/01/21 13:58:11 INFO mlflow.tracking._tracking_service.client: 🧪 View exp

In [15]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'XGB-Smote'.
2025/01/21 13:59:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.
2025/01/21 13:59:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: http://localhost:5001/#/experiments/627497453440672367/runs/1fcbb9f95521410c87f6eea10ecc39db.
2025/01/21 13:59:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/627497453440672367.


In [65]:
logged_model = 'runs:/0636cbd2e6cd44a18ef1dee77d10dba9/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [16]:
# Define the source model URI and target model name
source_model_name = 'XGB-Smote'
target_model_name = "anomaly-detection-prod"

# Retrieve the source model URI
latest_versions = client.get_latest_versions(name=source_model_name)
if not latest_versions:
    raise ValueError(f"No versions found for model '{source_model_name}'.")

# Use the latest version for the given stage
source_version = latest_versions[0].version
source_model_uri = f"models:/{source_model_name}/{source_version}"

# Copy the model version to the target model
client.copy_model_version(
    src_model_uri=source_model_uri,
    dst_name=target_model_name
)

print(f"Copied model version {source_version} of '{source_model_name}' to '{target_model_name}'.")


Copied model version 1 of 'XGB-Smote' to 'anomaly-detection-prod'.


Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


In [ ]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5001")



import mlflow
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Generate a 3-class classification problem
X, y = datasets.make_classification(
    n_samples=1000,
    class_sep=0.5,
    random_state=42,
    n_classes=3,
    n_informative=3,
)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Enable autolog
mlflow.autolog()  # or mlflow.sklearn.autolog()

# Initialize the classifier with n_estimators=200 and max_depth=10
clf = RandomForestClassifier(n_estimators=200, max_depth=10)

# Fit the classifier to the data.
clf.fit(X_train, y_train)

# Score the model on the data
clf.score(X_val, y_val)

# Visualize the automatically logged run results to validate what we recorded
mlflow.last_active_run()

2025/01/21 14:25:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/01/21 14:25:49 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.1, but the installed version is 2.1.3. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2025/01/21 14:25:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/01/21 14:25:50 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fe5feb46ef4b43df8eec55d6244b30c4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/01/21 14:25:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run inquisitive-cod-958 at: http://localhost:5001/#/experiments/627497453440672367/runs/fe5feb46ef4b43df8eec55d6244b30c4.
2025/01/21 14:25:54 INFO mlflow.tracking._

<Run: data=<RunData: metrics={'RandomForestClassifier_score_X_val': 0.705,
 'training_accuracy_score': 0.995,
 'training_f1_score': 0.9950071103659097,
 'training_log_loss': 0.33216231489218195,
 'training_precision_score': 0.9950746268656716,
 'training_recall_score': 0.995,
 'training_roc_auc': 0.9999085300191095,
 'training_score': 0.995}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'class_weight': 'None',
 'criterion': 'gini',
 'max_depth': '10',
 'max_features': 'sqrt',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '200',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': 'None',
 'verbose': '0',
 'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestClassifier',
 'estimator_name': 'RandomForestClassifier',
 'mlflow.autologging': 'sklearn',
 'mlflow.log-model.history